In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import ChatPromptTemplate
from langchain.load import dumps, loads
from operator import itemgetter
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.docstore.document import Document


def get_csv_files(directory):
    """
    Get all files ending with .csv in the specified directory.

    Parameters:
    - directory: str, the directory path where the files are located.

    Returns:
    - List of strings, containing the file paths of all .csv files in the directory.
    """
    csv_files = []
    for file in os.listdir(directory):
        if file.endswith(".csv"):
            csv_files.append(os.path.join(directory, file))
            
    # print(csv_files)
    return csv_files

def get_comments():
    files = get_csv_files("./llm")
    comments = []
    for file in files:
        df = pd.read_csv(file)
        comment_list = df["Comment"].tolist()
        
        comments.append(Document(page_content="\n".join(comment_list), metadata={"video": file}))
    return comments
def populate_db():
    comments = get_comments()
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=300, chunk_overlap=50
    )
    
    splits = text_splitter.split_documents(comments)
    print(splits)

    # # # Add to vectorDB
    # my_comments_vec_store = Chroma.from_documents(
    #     documents=splits,
    #     collection_name="my_comments",
    #     embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    #     persist_directory="./myCommentsdb"
    # )
    # my_comment_retriever = my_comments_vec_store.as_retriever()
    # return my_comment_retriever
# Load environment variables from .env file
load_dotenv()

gemini_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(google_api_key=gemini_key,model="gemini-pro",
                 temperature=0.7, top_p=0.85)

def load_from_db():
    db = Chroma(collection_name="my_comments",
        persist_directory="./myCommentsdb", embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
    retriever = db.as_retriever()
    return retriever



# # Multi Query: Different Perspectives
# template = """You are an AI language model assistant. Your task is to generate five 
# different versions of the given user question to retrieve relevant documents from a vector 
# database. By generating multiple perspectives on the user question, your goal is to help
# the user overcome some of the limitations of the distance-based similarity search. 
# Provide these alternative questions separated by newlines. Original question: {question}"""
# prompt_perspectives = ChatPromptTemplate.from_template(template)

# from langchain_core.output_parsers import StrOutputParser
# # from langchain_openai import ChatOpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI

# generate_queries = (
#     prompt_perspectives 
#     | ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7, top_p=0.85)
#     | StrOutputParser() 
#     | (lambda x: x.split("\n"))
# )


# def get_unique_union(documents: list[list]):
#     """ Unique union of retrieved docs """
#     # Flatten list of lists, and convert each Document to string
#     flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
#     # Get unique documents
#     unique_docs = list(set(flattened_docs))
#     # Return
#     return [loads(doc) for doc in unique_docs]

# # Retrieve
# question = "What do my audience love in my videos?"
# retrieval_chain =  retriever.map() | get_unique_union
# docs = retrieval_chain.invoke({"question":question})
template = """Answer the following question based on this context as a branding expert and a social media influencer advisor with the hope to help the user grow his or her social media handles.:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def chat(question):
    retriever = populate_db()
    # retriever = load_from_db()
    rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
    response = rag_chain.invoke(question)
    return response

if __name__ == "__main__":
    print(chat("do i talk fast?"))

[Document(page_content="What events did we miss?\nAccident case😢\nyou are baised,timle Rabi le yetro kand haru machayeko ra Dp aryal le manpower ko naam ma manche becheko batayenau,you are cognitive baised,aba haami le bhujyau ki RASAPA vanda raprapa 1000 times best rahexa\nMr beast 👀\n2081 ma ni chaiyooo same thing 😊\nAru kura ta thikai ho but it would have been better if you didn't include the indian news clips speaking in hindi. Tesko satta Nepali news clips nai haleko bhaye hunthyo.\nTeacher protest\nFake Bhutanese Refugee Scam\nWorld Cup qualified\nFormer King Gyanendra ko sakriyata xutyo\n\u200b@@PriyansW🎉\n1700+2...\nBipin Joshi \n3out1 mayor\nRam Prasad who died by fire on it \n5.4 Earthquake\nTeachers right ....\nA man det himself on fire as a sigh of protest\nFake Bhutanese Refugee Scam\nFake Bhutanese Refugee Scam\nWe deserve a long video and it would be better if you arrange the events according to the date. It's all mismatch. You can do better!!", metadata={'video': './llm

TypeError: unsupported operand type(s) for |: 'NoneType' and 'function'

In [24]:
# retriver = populate_db()

retriver = load_from_db()
retriver.get_relevant_documents("do i talk fast")

[Document(page_content='।'),
 Document(page_content='।'),
 Document(page_content='।'),
 Document(page_content='उर्जा मिलेको छ ।')]

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import ChatPromptTemplate
from langchain.load import dumps, loads
from operator import itemgetter
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader 

# def get_csv_files(directory):
#     """
#     Get all files ending with .csv in the specified directory.

#     Parameters:
#     - directory: str, the directory path where the files are located.

#     Returns:
#     - List of strings, containing the file paths of all .csv files in the directory.
#     """
#     csv_files = []
#     for file in os.listdir(directory):
#         if file.endswith(".csv"):
#             csv_files.append(os.path.join(directory, file))
#     return csv_files

# def get_comments():
#     files = get_csv_files("./llm")
#     comments = []
#     for file in files:
#         df = pd.read_csv(file)
#         comments.append("\n".join(df["Comment"].tolist()))
        
#     text = "\n\n\n".join(comments)
#     with open("text.txt", "w") as f:
#         f.wrtie(text)

def populate_db():
    
    loader = TextLoader("text.txt")
    docs = loader.load()

    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    # # # Add to vectorDB
    my_comments_vec_store = Chroma.from_documents(
        documents=splits,
        collection_name="my_comments",
        embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
        persist_directory="./myCommentsdb"
    )
    my_comment_retriever = my_comments_vec_store.as_retriever()
    return my_comment_retriever
# Load environment variables from .env file
load_dotenv()

gemini_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(google_api_key=gemini_key,model="gemini-pro",
                 temperature=0.7, top_p=0.85)

def load_from_db():
    db = Chroma(collection_name="my_comments",
        persist_directory="./myCommentsdb", embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
    retriever = db.as_retriever()
    return retriever



# # Multi Query: Different Perspectives
# template = """You are an AI language model assistant. Your task is to generate five 
# different versions of the given user question to retrieve relevant documents from a vector 
# database. By generating multiple perspectives on the user question, your goal is to help
# the user overcome some of the limitations of the distance-based similarity search. 
# Provide these alternative questions separated by newlines. Original question: {question}"""
# prompt_perspectives = ChatPromptTemplate.from_template(template)

# from langchain_core.output_parsers import StrOutputParser
# # from langchain_openai import ChatOpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI

# generate_queries = (
#     prompt_perspectives 
#     | ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7, top_p=0.85)
#     | StrOutputParser() 
#     | (lambda x: x.split("\n"))
# )


# def get_unique_union(documents: list[list]):
#     """ Unique union of retrieved docs """
#     # Flatten list of lists, and convert each Document to string
#     flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
#     # Get unique documents
#     unique_docs = list(set(flattened_docs))
#     # Return
#     return [loads(doc) for doc in unique_docs]

# # Retrieve
# question = "What do my audience love in my videos?"
# retrieval_chain =  retriever.map() | get_unique_union
# docs = retrieval_chain.invoke({"question":question})
template = """Answer the following question based on this context as a branding expert and a social media influencer advisor with the hope to help the user grow his or her social media handles.:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def chat(question):
    retriever = populate_db()
    # retriever = load_from_db()
    rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
    response = rag_chain.invoke(question)
    return response

if __name__ == "__main__":
    print(chat("do i talk fast?"))

The provided context does not contain any information about whether you talk fast or not, so I cannot answer this question from the provided context.


In [34]:
retriever = load_from_db()
retriever.get_relevant_documents("your talking speed is high")

[Document(page_content='।'),
 Document(page_content='।'),
 Document(page_content='।'),
 Document(page_content='उर्जा मिलेको छ ।')]

In [2]:
import re
def preprocess_text(text):
    try:
        # Remove emojis
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)

        # Remove mentioned users (@username)
        text = re.sub(r'@\w+', '', text)

        # Remove numbers
        text = re.sub(r'\d+', '', text)

        # Remove any remaining non-alphanumeric characters except whitespace
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Strip the text
        text = text.strip()

        # Remove non-English words
        # english_text = ' '.join(word for word in text.split() if word.lower() in english_words)
        # print(english_text)
        # return english_text
        return text
    except Exception as e:
        print("An error occurred during text preprocessing:", e)
        return ""


In [3]:
import pandas as pd
df = pd.read_csv("llm/competitor/if_i_started_a_yt_channel_in_24.csv")
comments = "\n".join(df["Comment"].tolist())
comments = preprocess_text(comments)
comments = comments.replace('\u2009', '')

In [4]:
with open("competitor.txt", "w") as f:
    f.write(comments)

In [2]:
import os
def get_csv_files(directory):
    """
    Get all files ending with .csv in the specified directory.

    Parameters:
    - directory: str, the directory path where the files are located.

    Returns:
    - List of strings, containing the file paths of all .csv files in the directory.
    """
    csv_files = []
    for file in os.listdir(directory):
        if file.endswith(".csv"):
            csv_files.append(os.path.join(directory, file))
            
    # print(csv_files)
    return csv_files


get_csv_files("./llm")

['./llm\\2080_in_6_min.csv',
 './llm\\announcement_how_to_youtube_course.csv',
 './llm\\biggest_plane_hijack_from_nepal.csv',
 './llm\\the_extra_ordinary_life_of_pushkar_shah.csv',
 './llm\\the_malaria_miracle.csv']

In [6]:
import pandas as pd
import json
with open("llm/topics.json", 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
print(df)


                                 name  \
0  announcement_how_to_youtube_course   
1                                 NaN   
2                                 NaN   
3                                 NaN   

                                              topics  
0  [course, ma, youtube, video, ko, guys, haru, t...  
1  {'name': 'the_malaria_miracle', 'topics': ['co...  
2  {'name': '2080_in_6_min', 'topics': ['ko', 'ma...  
3  {'name': 'the_extra_ordinary_life_of_pushkar_s...  


In [12]:
ls = [{"name":"announcement_how_to_youtube_course","topics":["course","ma","youtube","video","ko","guys","haru","think","like","ho","know","don","selling","money","just","ta","comment","people","videos","editing","yo","time","content","channel","ani","make","paisa","nepal","courses","kura","afno","nepali","free","sab","xa","algorithm","lai","right","buy","le","ra","youtuber","unsubscribe","good","watch","kei","want","doing","banauna","xaina"]},{"topics":{"name":"the_malaria_miracle","topics":["course","ma","youtube","video","ko","guys","haru","think","like","ho","know","don","selling","money","just","ta","comment","people","videos","editing","yo","time","content","channel","ani","make","paisa","nepal","courses","kura","afno","nepali","free","sab","xa","algorithm","lai","right","buy","le","ra","youtuber","unsubscribe","good","watch","kei","want","doing","banauna","xaina"]}},{"topics":{"name":"2080_in_6_min","topics":["ko","ma","nepal","video","ni","le","nepali","haru","ho","ra","kura","xa","bro","yo","news","khai","ta","\u091c\u092f","tei","kam","\u0917\u0930","ramro","desh","comment","hamro","rabi","lai","cup","good","garney","world","vayo","din","bata","thank","na","just","parxa","ka","india","xii","xaina","baised","better","\u0939\u0930","best","missed","pani","hunxa","chai"]}},{"topics":{"name":"the_extra_ordinary_life_of_pushkar_shah","topics":["ko","ma","book","visa","nepal","le","video","ni","lai","like","dai","nepali","travel","read","ho","country","puskar","na","shah","ra","nai","know","jay","jasto","politics","bela","xa","tara","bata","bro","story","tyo","man","pani","countries","brother","channel","daju","did","respect","pushkar","ultima","passport","videos","ta","think","bhik","world","haina","maile"]}}]
dic = {}
for i in ls:
    print(i)
    # dic[i["name"]] = i["topics"]

{'name': 'announcement_how_to_youtube_course', 'topics': ['course', 'ma', 'youtube', 'video', 'ko', 'guys', 'haru', 'think', 'like', 'ho', 'know', 'don', 'selling', 'money', 'just', 'ta', 'comment', 'people', 'videos', 'editing', 'yo', 'time', 'content', 'channel', 'ani', 'make', 'paisa', 'nepal', 'courses', 'kura', 'afno', 'nepali', 'free', 'sab', 'xa', 'algorithm', 'lai', 'right', 'buy', 'le', 'ra', 'youtuber', 'unsubscribe', 'good', 'watch', 'kei', 'want', 'doing', 'banauna', 'xaina']}
{'topics': {'name': 'the_malaria_miracle', 'topics': ['course', 'ma', 'youtube', 'video', 'ko', 'guys', 'haru', 'think', 'like', 'ho', 'know', 'don', 'selling', 'money', 'just', 'ta', 'comment', 'people', 'videos', 'editing', 'yo', 'time', 'content', 'channel', 'ani', 'make', 'paisa', 'nepal', 'courses', 'kura', 'afno', 'nepali', 'free', 'sab', 'xa', 'algorithm', 'lai', 'right', 'buy', 'le', 'ra', 'youtuber', 'unsubscribe', 'good', 'watch', 'kei', 'want', 'doing', 'banauna', 'xaina']}}
{'topics': {'na

In [14]:
topics = {"announcement_how_to_youtube_course":['course', 'ma', 'youtube', 'video', 'ko', 'guys', 'haru', 'think', 'like', 'ho', 'know', 'don', 'selling', 'money', 'just', 'ta', 'comment', 'people', 'videos', 'editing', 'yo', 'time', 'content', 'channel', 'ani', 'make', 'paisa', 'nepal', 'courses', 'kura', 'afno', 'nepali', 'free', 'sab', 'xa', 'algorithm', 'lai', 'right', 'buy', 'le', 'ra', 'youtuber', 'unsubscribe', 'good', 'watch', 'kei', 'want', 'doing', 'banauna', 'xaina'],
"the_malaria_miracle":  ['course', 'ma', 'youtube', 'video', 'ko', 'guys', 'haru', 'think', 'like', 'ho', 'know', 'don', 'selling', 'money', 'just', 'ta', 'comment', 'people', 'videos', 'editing', 'yo', 'time', 'content', 'channel', 'ani', 'make', 'paisa', 'nepal', 'courses', 'kura', 'afno', 'nepali', 'free', 'sab', 'xa', 'algorithm', 'lai', 'right', 'buy', 'le', 'ra', 'youtuber', 'unsubscribe', 'good', 'watch', 'kei', 'want', 'doing', 'banauna', 'xaina'],
"2080_in_6_min": ['ko', 'ma', 'nepal', 'video', 'ni', 'le', 'nepali', 'haru', 'ho', 'ra', 'kura', 'xa', 'bro', 'yo', 'news', 'khai', 'ta', 'जय', 'tei', 'kam', 'गर', 'ramro', 'desh', 'comment', 'hamro', 'rabi', 'lai', 'cup', 'good', 'garney', 'world', 'vayo', 'din', 'bata', 'thank', 'na', 'just', 'parxa', 'ka', 'india', 'xii', 'xaina', 'baised', 'better', 'हर', 'best', 'missed', 'pani', 'hunxa', 'chai'],
"the_extra_ordinary_life_of_pushkar_shah": ['ko', 'ma', 'book', 'visa', 'nepal', 'le', 'video', 'ni', 'lai', 'like', 'dai', 'nepali', 'travel', 'read', 'ho', 'country', 'puskar', 'na', 'shah', 'ra', 'nai', 'know', 'jay', 'jasto', 'politics', 'bela', 'xa', 'tara', 'bata', 'bro', 'story', 'tyo', 'man', 'pani', 'countries', 'brother', 'channel', 'daju', 'did', 'respect', 'pushkar', 'ultima', 'passport', 'videos', 'ta', 'think', 'bhik', 'world', 'haina', 'maile']
}
topics_model = {}
for key in topics:
    topics_model[key] = " ".join(topics[key])
topics_model

{'announcement_how_to_youtube_course': 'course ma youtube video ko guys haru think like ho know don selling money just ta comment people videos editing yo time content channel ani make paisa nepal courses kura afno nepali free sab xa algorithm lai right buy le ra youtuber unsubscribe good watch kei want doing banauna xaina',
 'the_malaria_miracle': 'course ma youtube video ko guys haru think like ho know don selling money just ta comment people videos editing yo time content channel ani make paisa nepal courses kura afno nepali free sab xa algorithm lai right buy le ra youtuber unsubscribe good watch kei want doing banauna xaina',
 '2080_in_6_min': 'ko ma nepal video ni le nepali haru ho ra kura xa bro yo news khai ta जय tei kam गर ramro desh comment hamro rabi lai cup good garney world vayo din bata thank na just parxa ka india xii xaina baised better हर best missed pani hunxa chai',
 'the_extra_ordinary_life_of_pushkar_shah': 'ko ma book visa nepal le video ni lai like dai nepali tra

In [15]:
df = pd.DataFrame(topics_model)
df.to_csv("topics.csv", index = False)

ValueError: If using all scalar values, you must pass an index

In [ ]:
import pandas as pd

df = pd.read_csv